In [19]:
%%bash 
apt-get install -y unzip
wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
unzip -o bank-additional.zip


Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-23+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


--2020-08-17 10:27:51--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.184.137
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.184.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

     0K .......... .......... .......... .......... .......... 11%  412K 1s
    50K .......... .......... .......... .......... .......... 23%  786K 1s
   100K .......... .......... .......... .......... .......... 35% 86.6M 0s
   150K .......... .......... .......... .......... .......... 47% 93.4M 0s
   200K .......... .......... .......... .......... .......... 59%  794K 0s
   250K .......... .......... .......... ...

In [20]:
!ls -al 

total 533436
drwx------ 9 root nogroup      6144 Aug 17 10:27 .
drwxr-xr-x 1 root root           61 Aug 17 01:59 ..
drwxr-xr-x 4 root root         6144 Aug 17 02:02 .cache
drwxr-xr-x 3 root root         6144 Aug 17 02:02 .config
drwxr-xr-x 2 root root         6144 Aug 17 03:48 .ipynb_checkpoints
drwxr-xr-x 5 root root         6144 Aug 17 01:59 .ipython
drwxr-xr-x 3 root root         6144 Aug 17 01:55 .jupyter
drwxr-xr-x 3 root root         6144 Aug 17 01:54 .local
-rw-r--r-- 1 root root          180 Aug 17 03:48 .wget-hsts
-rw-r--r-- 1 root root          111 Aug 17 01:54 .yarnrc
-rw-r--r-- 1 root root       179227 Aug 17 08:42 auto-pilot-fraud-detection.ipynb
-rw-r--r-- 1 root root    124692809 Aug 17 03:55 automl-train.csv
drwxr-xr-x 2 root root         6144 Jan 15  2020 bank-additional
-rw-r--r-- 1 root root       432828 Jan 15  2020 bank-additional.zip
-rw-r--r-- 1 root root    150828752 Mar 23  2018 creditcard.csv
-rw-r--r-- 1 root root     69155632 Mar 29  2019 creditcardfraud.zip

In [28]:

import pandas as pd

data = pd.read_csv('./bank-additional/bank-additional-full.csv')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [29]:
import sagemaker

prefix = 'sagemaker/tutorial-autopilot/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="./bank-additional/bank-additional-full.csv", key_prefix=prefix)
print(uri)



s3://sagemaker-us-east-1-230755935769/sagemaker/tutorial-autopilot/input/bank-additional-full.csv


In [30]:
import boto3, sys

ep_name = 'tutorial-autopilot-best-model'
sm_rt = boto3.Session().client('runtime.sagemaker')

tn=tp=fn=fp=count=0

with open('bank-additional/bank-additional-full.csv') as f:
    lines = f.readlines()
    for l in lines[1:2000]:   # Skip header
        l = l.split(',')      # Split CSV line into features
        label = l[-1]         # Store 'yes'/'no' label
        l = l[:-1]            # Remove label
        l = ','.join(l)       # Rebuild CSV line without label
                
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, 
                                         ContentType='text/csv',       
                                         Accept='text/csv', Body=l)

        response = response['Body'].read().decode("utf-8")
        #print ("label %s response %s" %(label,response))

        if 'yes' in label:
            # Sample is positive
            if 'yes' in response:
                # True positive
                tp=tp+1
            else:
                # False negative
                fn=fn+1
        else:
            # Sample is negative
            if 'no' in response:
                # True negative
                tn=tn+1
            else:
                # False positive
                fp=fp+1
        count = count+1
        if (count % 100 == 0):   
            sys.stdout.write(str(count)+' ')
            
print ("Done")

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tp/(tp+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint tutorial-autopilot-best-model of account 230755935769 not found.